In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [ ]:
filepath = f'fake reviews dataset.csv'
df = pd.read_csv(filepath)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['review'] = df['text_'].apply(preprocess_text)
X = df['review']
y = df['label']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def plot_confusion_matrix(cm, model_name):
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [3]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(),
        'SVM': SVC(),
        'Random Forest': RandomForestClassifier(),
        'XGBoost': XGBClassifier()
    }

    accuracies = {}

    for model_name, model in models.items():
        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        accuracies[model_name] = accuracy_score(y_test, y_pred)

        print(f'{model_name} Classification Report:\n{classification_report(y_test, y_pred)}')
        
        cm = confusion_matrix(y_test, y_pred)
        print(f'{model_name} Confusion Matrix:\n{cm}')
        plot_confusion_matrix(cm, model_name)

    return accuracies

model_accuracies = train_and_evaluate_models(X_train, X_test, y_train, y_test)

In [ ]:
X_train_tensor = torch.FloatTensor(X_train_tfidf.toarray()).to(device)
X_test_tensor = torch.FloatTensor(X_test_tfidf.toarray()).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

class LSTMModel(nn.Module):
    def _init_(self, input_size, hidden_size=250, num_layers=1):
        super(LSTMModel, self)._init_()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Reshape input for LSTM [batch_size, seq_length, input_size]
        x = x.unsqueeze(1)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
       
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        out = self.sigmoid(out)
        return out.squeeze()